In [47]:
import itertools

class PitcherPuzzle:
    """
    Solves the Pitcher Problem where water must be poured between a series of
    pitchers to reach a certain state. 
    
    Example
    -------
    Three unmarked pitchers have capacities 10, 7 and 3.
    The first pitcher contains 10, the other two are empty.
    Pour water between the pitchers so that two pitchers hold 5 each.
    
    In this case, the starting state is (10,0,0).
    After 1 pour there are two possible states:
    1. Pour from the 10 into the 7 -> (3,7,0)
    2. Pour from the 10 into the 3 -> (7,0,3)
    
    Given a desired state (5,5,0) we want to find the solution which uses
    the fewest pours.
    
    Algorithm
    ---------
    We investigate all possible states using breadth-first search, stopping
    when we reach a state already visited, and keep track of the 'path' 
    (the series of pours) taken to reach each state. 
    
    """
    def __init__(self, 
                 pitchers: tuple[float, ...], 
                 start_state: tuple[float, ...]
                 ):
        if len(start_state) != len(pitchers):
            raise ValueError(
                "the start state must be the same length as the number of pitchers"
                )
        
        self.pitchers: tuple[float, ...] = pitchers
        self.n_pitchers: int = len(pitchers)
        self.states_visited: list[tuple[float, ...]] = [start_state]
        self.paths: list[list[tuple[int, ...]]] = [
            [(0,)]
        ]
        self.solve()
  
    def next_states(self, state) -> set[tuple[float, ...]]:
        _states = set()
        for i in range(self.n_pitchers):
            if state[i] != 0:
                for j in range(self.n_pitchers):
                    if i == j:
                        continue

                    _pour_size = min(self.pitchers[j] - state[j], state[i])
                    if _pour_size == 0:
                        continue
                    
                    _new_state = list(state)
                    _new_state[i] = state[i] - _pour_size
                    _new_state[j] = state[j] + _pour_size

                    _states.add(tuple(_new_state))
    
        return _states

    @property
    def paths_list(self):
        _l = []
        for _paths in self.paths:
            _l.extend(_paths)
        return _l

    def state_index(self, state):
        if state in self.states_visited:
            return self.states_visited.index(state)
        return -1

    def advance_solution(self):
        _new_paths = []
        for _p in self.paths[-1]:
            next_states = self.next_states(self.states_visited[_p[-1]]) - \
                set(self.states_visited)
            if len(next_states) == 0:
                continue

            for _s in next_states:
                _new_path = tuple(list(_p) + [len(self.states_visited)])
                _new_paths.append(_new_path)
                self.states_visited.append(_s)
      
        if len(_new_paths) == 0:
            return 0

        self.paths.append(_new_paths)
        return 1

    def solve(self):
        _code = 1
        while _code:
            _code = self.advance_solution()

    def path_to_state(self, state: tuple[float, ...], ordered: bool = False):
        if ordered:
            if len(state) > self.n_pitchers:
                raise ValueError("Too many pitchers")
            if len(state) < self.n_pitchers:
                raise ValueError("Too few pitchers")
            states = {state}
        else:
            if len(state) > self.n_pitchers:
                raise ValueError("Too many pitchers")
            if len(state) < self.n_pitchers:
                state = tuple(list(state) + [0]*(self.n_pitchers - len(state)))
            states = set(itertools.permutations(state))
   
        indices = [self.state_index(_s) for _s in states]
        indices = [_ for _ in indices if _ != -1]
        if len(indices) == 0:
            print("Solution does not exist")
            return
        
        _paths = []
        for _indx in indices:
            _paths.extend([_p for _p in self.paths_list if _p[-1]==_indx])
        _paths = sorted(_paths, key=len)
  
        return [self.states_visited[i] for i in _paths[0]]
    
    def print_path_to_state(self, state: tuple[float, ...], ordered: bool = False):
        path = self.path_to_state(state=state, ordered=ordered)
        n = len(path) - 1
        print(f"Solution in {n} pours:\n")
        
        pad = len(str(n))
        for i, s in enumerate(path):
            print(f"{i:<{pad}}: {s}")

In [67]:
p = PitcherPuzzle((10,7,3), (10,0,0))

In [69]:
p.print_path_to_state((5,5))

Solution in 9 pours:

0: (10, 0, 0)
1: (3, 7, 0)
2: (3, 4, 3)
3: (6, 4, 0)
4: (6, 1, 3)
5: (9, 1, 0)
6: (9, 0, 1)
7: (2, 7, 1)
8: (2, 5, 3)
9: (5, 5, 0)
